> https://leetcode.com/problems/combinations/

In [39]:
function _combine(nums, k) {
    if (k === 1) return nums.map(v => [v]);
    if (k === 0) return [];
    
    let num_idx = 0;
    let out = [];
    for (let num of nums) {
        let remaining_nums = [...nums.slice(0, num_idx), ...nums.slice(num_idx + 1, nums.length)];
        let combos = _combine(remaining_nums, k - 1);
        combos = combos.map(c => [num, ...c]);
        out = [...out, ...combos];
        num_idx += 1;
    }
    return out;
}

function combine(n, k) {
    let nums = [...Array(n).keys()].map(v=> v + 1);
    return _combine(nums, k);
}

In [36]:
_combine([1,2,3], 2)

[ [ 1, 2 ], [ 1, 3 ], [ 2, 1 ], [ 2, 3 ], [ 3, 1 ], [ 3, 2 ] ]

In [37]:
[1,2,3].slice(0,3)

[ 1, 2, 3 ]

In [38]:
_combine([1,2,3], 3)

[ [ 1, 2, 3 ],
  [ 1, 3, 2 ],
  [ 2, 1, 3 ],
  [ 2, 3, 1 ],
  [ 3, 1, 2 ],
  [ 3, 2, 1 ] ]

* Implementation stalled for a while because I had accidentally left out a `let` and thus had a global variable in the wrong place (might want to stick to `use strict;` going forward).
* The function loops through a length $n$ array, recursively calling itself with the $k - 1$. Each subcall itself requires $n$ iterations. The call tree stops when $k = 1$, but since each subsequent layer has one fewer number to consider, the actual number of bottom calls is $k!$. Additionally, each bottom call is the subject of a call chain of length $k$. That implies $k(k!) \approx k!$ total calls. Each call performs $n$ work, where $n$ is the length of the list, because it must translate it to a new array and store that. Thus the overall runtime is $O(n(k!))$.
* On review, this actually solves for permutations. Whoops. Adapt the code to solve for combinations.

In [40]:
function _combine(nums, k) {
    if (k === 1) return nums.map(v => [v]);
    if (k === 0) return [];
    
    let num_idx = 0;
    let out = [];
    for (let num of nums) {
        let remaining_nums = nums.slice(num_idx + 1, nums.length);
        let combos = _combine(remaining_nums, k - 1);
        combos = combos.map(c => [num, ...c]);
        out = [...out, ...combos];
        num_idx += 1;
    }
    return out;
}

function combine(n, k) {
    let nums = [...Array(n).keys()].map(v=> v + 1);
    return _combine(nums, k);
}

In [41]:
combine(4, 2)

[ [ 1, 2 ], [ 1, 3 ], [ 1, 4 ], [ 2, 3 ], [ 2, 4 ], [ 3, 4 ] ]

* This is a very small modification: instead of allowing for all other numbers on the recursive call, we allow only for those numbers which lie to the right of the current one.
* This means that instead of $k!$ bottom calls have have $n \choose k$ bottom calls, which nevertheless subtends to $n!$ calls, so the $O$ speed is unchanged.

> https://leetcode.com/problems/combination-sum/

I already solved this one three months ago.

In [83]:
function combinationSum(candidates, k) {
    return _combinationSum(candidates, k, [], []);
}

function _combinationSum(nums, k, prior_nums, solutions=[]) {
    if (nums.length === 0) return solutions;
    
    let prior_sum = prior_nums.length !== 0 ?
        prior_nums.reduce((a, b) => a + b) : 0;
    let num_idx = 0;
    for (let num of nums) {
        if (prior_sum + num === k) {
            solutions.push([...prior_nums, num]);
        } else if (prior_sum + num < k) {
            solutions = _combinationSum(
                nums.slice(num_idx + 1, nums.length), k,
                [...prior_nums, num], solutions
            );
        }
        num_idx += 1;
    }
    return solutions;
}

In [84]:
_combinationSum([1,2,3,4], 4, [], [])

[ [ 1, 3 ], [ 4 ] ]

In [85]:
combinationSum([2,3,6,7], 7)

[ [ 7 ] ]

* Welp! This solves for the case where numbers cannot be repeated, but the problem allows repeats.

In [86]:
function combinationSum(candidates, k) {
    return _combinationSum(candidates, k, [], []);
}

function _combinationSum(nums, k, prior_nums, solutions=[]) {
    if (nums.length === 0) return solutions;
    
    let prior_sum = prior_nums.length !== 0 ?
        prior_nums.reduce((a, b) => a + b) : 0;
    let num_idx = 0;
    for (let num of nums) {
        if (prior_sum + num === k) {
            solutions.push([...prior_nums, num]);
        } else if (prior_sum + num < k) {
            solutions = _combinationSum(
                nums, k,
                [...prior_nums, num], solutions
            );
        }
        num_idx += 1;
    }
    return solutions;
}

In [87]:
combinationSum([2,3,6,7], 7)

[ [ 2, 2, 3 ], [ 2, 3, 2 ], [ 3, 2, 2 ], [ 7 ] ]

* This code solves for *permutations* with repeats, whilst we need *combinations* with repeats. Easily enough fixed using a map instead, however.
* Here's a much better solution in Python that was posted [to the forums](https://leetcode.com/problems/combination-sum/discuss/16554/Share-My-Python-Solution-beating-98.17):

In [3]:
def combinationSum(candidates, target):
    result = []
    candidates = sorted(candidates)
    
    def dfs(remain, stack):
        if remain == 0:
            result.append(stack)
            return 

        for item in candidates:
            if item > remain: break
            if stack and item < stack[-1]: continue
            else:
                dfs(remain - item, stack + [item])
    
    dfs(target, [])
    return result

In [18]:
combinationSum([2,3,6,7], 7)

[ [ 3, 2, 2 ], [ 7 ] ]

* This solution is the same as my solution (albeit much less verbose) with a key difference. It sorts the values beforehand. Then, during iteration, it only branches recursively if the number it is on is equal to or greater than the last number on the stack. This ensures that the values are iterated over in ascending shape, e.g. `[2,2,3]`, and neatly handles the "permutations revisiting the same solution" problem, which I couldn't immediately think of a way to avoid.
* The use of the prior sort also allows for backtracking: exiting the task loop when the remaining numbers are all too large. I did this too in my solution, but on a case-by-case basis, e.g. I did not break out completely but only stopped work on a case-by-case basis. This is thus only a linear time optimization.
* Implemented in JS below.

In [17]:
function combinationSum(candidates, k) {
    return _combinationSum(candidates.sort((a, b) => a - b), k, [], []);
}

function _combinationSum(nums, k, prior_nums, solutions=[]) {
    if (nums.length === 0) return solutions;
    
    let prior_sum = prior_nums.length !== 0 ?
        prior_nums.reduce((a, b) => a + b) : 0;
    let num_idx = 0;
    for (let num of nums) {
        if (prior_sum + num === k) {
            solutions.push([...prior_nums, num]);
        } else if ((prior_sum + num < k) &&
                   ((prior_nums.length > 0) && (num >= prior_nums[prior_nums.length - 1]))) {
        } else if ((prior_sum + num < k)) {
            solutions = _combinationSum(
                nums, k,
                [...prior_nums, num], solutions
            );
        } else {
            break;
        }
        num_idx += 1;
    }
    return solutions;
}

In [4]:
combinationSum([2,3,6,7], 7)

[[2, 2, 3], [7]]

* There are $n!$ bottom calls (assuming, as in the worst case, no backtracking), each of which is up to $n$ in length. Each call performs constant (amortized) work. There is also a sort, which is $O(n \log{n})$. The total cost is $O(n\log{n} + n(n!)) \approx O(n!)$.

> https://leetcode.com/problems/combination-sum-ii/

In [92]:
function combinationSum2(candidates, k) {
    return _combinationSum2(candidates.sort((a, b) => a - b), k, [], []);
}

function _combinationSum2(nums, k, prior_nums, solutions=[]) {
    if (nums.length === 0) return solutions;
    let prior_sum = prior_nums.length === 0 ? 0 : prior_nums.reduce((a, b) => a + b);
    
    let num_idx = 0;
    let numbers_already_seen = new Set();
    
    for (let num of nums) {
        if (numbers_already_seen.has(num)) {
            //
        } else if (prior_sum + num === k) {
            solutions.push([...prior_nums, num]);
        } else if (prior_sum + num > k) {
            break
        } else if ((prior_nums.length === 0) &&
                   (prior_nums[prior_nums.length - 1] === num)) {
            console.log(nums, k, prior_nums, solutions, num);
        } else {
            solutions = _combinationSum2(
                nums.slice(num_idx + 1, nums.length), k,
                [...prior_nums, num], solutions
            );
        }
        numbers_already_seen.add(num);
        num_idx += 1;
    }
    
    return solutions;
}

In [88]:
combinationSum2([10,1,2,7,6,1,5], 8)

[ [ 1, 1, 6 ], [ 1, 2, 5 ], [ 1, 7 ], [ 2, 6 ] ]

In [89]:
combinationSum2([10,1,2,7,6,5], 8)

[ [ 1, 2, 5 ], [ 1, 7 ], [ 2, 6 ] ]

In [93]:
combinationSum2([3,1,3,5,1,1], 8)

[ [ 1, 1, 1, 5 ], [ 1, 1, 3, 3 ], [ 3, 5 ] ]

* This one took me much longer than I care to admit. Sigh.
* I figured out relatively quickly what the trick to doing this one is on paper. Order the elements, and then iterate them in the same backtracking, post-nodal manner as in the previous combinatorical problem. This time, however, instead of simply picking the next value from the list for recursive investigation, we pick the next *unique* value from the list for recursive investigation. In other words, if we have an array `[1,1,1,2,3,...]` the elements at the top level of the iteration are `[1,2,3,..]`.

  Although I grasped how this could be done conceptually easily enough, I struggled at implementing this logic in code (e.g. the mind-code connection), and placed in the wrong place in the iteration order, and struggled to understand where the right place to put it was.
  
  I needed to do more paper analysis! Do a better job of figuring out the cases on paper before implementation!

> https://leetcode.com/problems/letter-combinations-of-a-phone-number/

In [11]:
let dmap = new Map([
    ['2', 'abc'],
    ['3', 'def'],
    ['4', 'ghi'],
    ['5', 'jkl'],
    ['6', 'mno'],
    ['7', 'pqrs'],
    ['8', 'tuv'],
    ['9', 'wxyz']
]);

In [13]:
function letterCombinations(digits) {
    if (digits.length === 0) return [];
    if (digits.length === 1) return dmap.get(digits).split('');
    
    let [first, remainder] = [digits.toString()[0],
                              digits.toString().slice(1)];
    let lCs = letterCombinations(remainder);
    let fdOpts = dmap.get(first).split('');
    let res = [];
    for (let lC of lCs) {
        for (let fdOpt of fdOpts) {
            res.push(fdOpt + lC);
        }
    }
    return res;
}

* Easy enough.
* This function is $O(3^n)$ amortized time. Each call creates three recursive branches, except in the case that the 7 key is hit, but the 7 key is hit rarely enough that we can just say that the branching factor is three. The number of times branching is required is the length of the string. There are $n$ prior calls leading up to each bottom call, but since the functin does constant work this does not affect the O-speed. The total time cost is $O(3^n)$.
* This function uses $n$ stack space. At most $3^n$ space is required for the array constructed for output. Therefore this function has $O(3^n)$ memory cost.

* Summary of this segment of problems. Permutations with and without replacement and combinations with and without replacement all have their own slightly different iteration strategies that you need to employ to get them. And, how to implement said strategies has to be intuitively obvious.

> https://leetcode.com/problems/two-sum/

In [25]:
function twoSum(nums, target) {
    let outset = new Set();
    for (let num of nums) {
        if (num <= target) {
            let remainder = target - num;
            if (outset.has(remainder)) return [remainder, num];
            else outset.add(num);
        }
    }
}

In [26]:
twoSum([1,2],3)

[ 1, 2 ]

* This is the first problem on Leetcode numerically. I had solved this one before, of course. This solution is $O(n)$, and requires $O(n)$ space.

> https://leetcode.com/problems/two-sum-ii-input-array-is-sorted/ (try two pointer method)

In [27]:
function twoSum(nums, target) {
    let [lp, rp] = [0, nums.length - 1];
    while (lp < rp) {
        if (nums[lp] + nums[rp] === target) {
            return [lp + 1, rp + 1];
        } else if (nums[lp] + nums[rp] > target) {
            rp -= 1;
        } else if (nums[lp] + nums[rp] < target) {
            lp += 1;
        }
    }
}

* This algorithm is still $O(n)$ but requires only $O(1)$ space. It'd be $O(n \log{n})$ if the input array wasn't mandated to be sorted ahead of time.

> https://leetcode.com/problems/3sum/

In [48]:
function threeSum(nums) {
    let first_idx = 0;
    let out = [];
    
    nums = nums.sort((a, b) => a - b);
    
    for (let num of nums) {
        if (first_idx > 0 && num === nums[first_idx - 1]) {
            first_idx += 1;
            continue;
        }
        
        let [second_idx, third_idx] = [first_idx + 1, nums.length - 1];
        
        while (second_idx < third_idx) {
            if (nums[second_idx] + nums[third_idx] + nums[first_idx] === 0) {
                out.push([nums[first_idx], nums[second_idx], nums[third_idx]]);
                
                while ((second_idx + 1 < third_idx) && nums[second_idx + 1] === nums[second_idx]) {
                    second_idx += 1;
                }
                second_idx += 1;

                while ((third_idx - 1 > second_idx) && nums[third_idx - 1] === nums[third_idx]) {
                    third_idx -= 1;
                }
                third_idx -= 1;
                
            } else if (nums[second_idx] + nums[third_idx] + nums[first_idx] < 0) {
                second_idx += 1;
            } else {
                third_idx -= 1;
            }
        }
        
        first_idx += 1;
    }
    
    return out;
}

In [49]:
threeSum([-1,0,1,2,-1,-4])

[ [ -1, -1, 2 ], [ -1, 0, 1 ] ]

* Another one I had previously solved three months ago.
* This algorithm is $O(n^2)$, as it iterates over the array (with a pointer) whilst iterating over the array (with a for loop). As we are appending to an output array, it requires $O(n)$ space.

> https://leetcode.com/problems/4sum/ (can you do this in better than O(n**3)?)

In [108]:
function fourSum(nums, target) {
    function skipMoveRight(nums, p_idx, max_idx) {
        let curr_v = nums[p_idx];
        while (p_idx < max_idx && curr_v === nums[p_idx + 1]) p_idx += 1;
        p_idx += 1;
        return p_idx;
    }
    
    function skipMoveLeft(nums, p_idx, min_idx) {
        let curr_v = nums[p_idx];
        while (p_idx > min_idx && curr_v === nums[p_idx - 1]) p_idx -= 1;
        p_idx -= 1;
        return p_idx;
    }
    
    let out = [];
    let [outer_left_p, outer_right_p] = [0, nums.length -1];
    let increment = 'left';
    nums = nums.sort((a, b) => a - b);
    
    while (outer_left_p < outer_right_p) {
        let [inner_left_p, inner_right_p] = [outer_left_p + 1, outer_right_p - 1];
        
        while (inner_left_p < inner_right_p) {
            let [a, b, c, d] = [
                nums[outer_left_p], nums[inner_left_p], nums[outer_right_p], nums[inner_right_p]
            ];
            let s = a + b + c + d;
            if (s < target) inner_left_p += 1;
            else if (s > target) inner_right_p -= 1;
            else if (s === target) {
                out.push([a, b, c, d]);
                inner_left_p = skipMoveRight(nums, inner_left_p, inner_right_p);
                inner_right_p = skipMoveLeft(nums, inner_right_p, inner_left_p);
            }
        }
        
        if (outer_left_p < outer_right_p - 3) {
            outer_left_p = skipMoveRight(nums, outer_left_p, outer_right_p - 3);
        } else if (outer_right_p > 3) {
            outer_right_p = skipMoveLeft(nums, outer_right_p, 3);
            outer_left_p = 0;
        } else {
            break;
        }
        
    }
    
    return out;
}

In [109]:
fourSum([1, 0, -1, 0, -2, 2], 0)

[ [ -2, -1, 2, 1 ], [ -2, 0, 2, 0 ], [ -1, 0, 1, 0 ] ]

In [110]:
fourSum([0, 0, 0, 0], 0)

[ [ 0, 0, 0, 0 ] ]

In [111]:
fourSum([0, 0, 0, 0], 1)

[]

In [112]:
fourSum([-3,-1,0,2,4,5], 0)

[ [ -3, -1, 4, 0 ] ]

In [113]:
fourSum([5,5,3,5,1,-5,1,-2], 4)

[ [ -5, 1, 5, 3 ] ]

* My initial idea, which turned out incorrect, was that you could get all of the cases if you incremented a left outer pointer, then a right outer pointer, in sequence.
* The correct implementation instead uses a for-style iterator (with the important exception that it skips over same-number values).
* This question took a long time to get a fully correct answer for besides that because of various small code errors. It's a pretty complex routine.
* This algorithm is $O(n^3)$, I don't see how you could do this faster.

> Implement BFS and DFS using stacks and queues. Learn how to put tuples into a stack (instead of just one item)

Stacks are LIFO, queues are FIFO. A good implementation of either has $O(1)$ inserts and $O(1)$ pop/dequeue operations. Stacks are easily enough implemented using an array, which has $O(1)$ amortized time for the built-in `pop` and `push` operations. A queue requires a singly linked list with two pointers: one pointing to the start, one at the end. Each insert moves the end pointer. Each dequeue moves the start pointer (using the pointer to next to find the new starting node).

Putting a tuple of elements in a stack is as simple as calling push multiple times?

The call stack for recursive BFS is an implicit stack. The recursive solution can be rewritten as an iterative one by using an explicit stack and a while loop that attempts to exhaust that stack.

DFS has two simultaneous queues or stacks, one with the current nodes and one with the next deepest layer of nodes. You can build up one while you exhaust the other.